# PROYECTO COVID-19

Imports necesarios para el correcto funcionamiento del notebook

In [9]:
!pip install pytest-shutil
!pip install opencv-python
!pip install imgaug
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import os
import shutil
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from skimage.feature import local_binary_pattern
%matplotlib inline
import time
from skimage.feature import hog
import re
from sklearn.multiclass import OneVsRestClassifier

  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4835 sha256=d3c95b6ea1685a5cc1a2b2b1b240117f46198fdf3ea5a1f75bff078b49c00852
  Stored in directory: C:\Users\julen\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built termcolor


# Primera aproximación del problema

Función que permite ordenar Strings alfanuméricos

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

Función que permite la lectura de imágenes a partir del directorio en el que estan alamacenadas:

In [3]:
def lectura(subdirectorio,fichero,path="Dataset/"):
    if(os.path.exists(path+subdirectorio)):
        listaImagenes = sorted_alphanumeric(os.listdir(path+subdirectorio))
        datos = []

        print("Leyendo imágenes del directorio: "+path+subdirectorio)
        for imagen in listaImagenes:
            datos.append(cv2.imread(path+subdirectorio+imagen,0))
        if(os.path.isfile(path+fichero)):
            with open(path+fichero) as fp: 
                salidas = fp.read().splitlines()
            return np.array(datos), np.array(salidas)
        else:
            print("¡Error! El fichero de salidas no existe")
    else:
        print("¡Error! El directorio no existe")

Definimos la funcion para leer imagenes y aumentar el tamaño de nuestro conjunto de datos, utilizando:
- Ruido Gaussiano
- Volteando verticalmente

In [4]:
def tratamiento(pathImagenes="Data/", dx=256, dy=256, noiseX=5, noiseY=10):
    if(os.path.exists(pathImagenes)):
        print("Tratando y generando las imágenes: "+pathImagenes)
        resultado = []
        listaImagenes = os.listdir(pathImagenes)
        
        #Realizar un data augmentation para aumentar los datos del dataset:
        gaussian_noise = iaa.AdditiveGaussianNoise(noiseX, noiseY)
        flip_vr=iaa.Fliplr(p=1.0)

        for imagen in listaImagenes:
            Xray = cv2.resize(cv2.imread(pathImagenes+imagen,0),(dx, dy))
            resultado.append(Xray)
            resultado.append(gaussian_noise.augment_image(Xray))
            resultado.append(flip_vr.augment_image(Xray))

        return resultado
    else:
        print("¡Error! El path especificado no ha sido encontrado")
        return None

Definimos una funcion para hacer el particionamiento de los datos en train, test y val

In [5]:
def particionamiento(listaDatos, train_percent=.6, validate_percent=.2, seed=0):
    np.random.seed(seed)
    listaTrain=[]
    listaTest=[]
    listaVal=[]
    y_train = []
    y_test = []
    y_val = []
    print("Generando el particionamiento...")
    for i,lista in enumerate(listaDatos):
        lista = np.array(lista)
        perm = np.random.permutation(len(lista))
        m = len(lista)
        train_end = int(train_percent * m)
        val_end = int(validate_percent * m) + train_end
        train = lista[perm[:train_end]]
        val = lista[perm[train_end:val_end]]
        test = lista[perm[val_end:]]
        listaTrain.extend(train)
        listaTest.extend(test)
        listaVal.extend(val)
        y_train.extend(list([i]*len(train)))
        y_test.extend(list([i]*len(test)))
        y_val.extend(list([i]*len(val)))
    
    return listaTrain, listaTest, listaVal, y_train, y_test, y_val

Esta función permite almacenar las imágenes leidas y tratasdas durante la fase de train:

In [6]:
def almacenaImagenes(train,test,val,y_train,y_test,y_val,path="Dataset/"):
    if(os.path.exists(path)):
        try:
            shutil.rmtree(path,ignore_errors=True)
            
        except OSError as e:
            print("¡Error! No se ha podido eliminar el directorio")
    print("Almacenando las imágenes en disco...")
    os.mkdir(path, 0o7777)
    os.mkdir(path+"train/",0o7777)
    os.mkdir(path+"val/",0o7777)
    os.mkdir(path+"test/",0o7777)
    
    f = open(path+"train.txt", "w")
    for i,imagen in enumerate(train):
        cv2.imwrite(path+"train/Train-"+str(i)+".png",imagen)
        f.write(str(y_train[i])+"\n")
    f.close()
    
    f = open(path+"test.txt", "w")
    for i,imagen in enumerate(test):
        cv2.imwrite(path+"test/Test-"+str(i)+".png",imagen)
        f.write(str(y_test[i])+"\n")
    f.close()
    
    f = open(path+"val.txt", "w")
    for i,imagen in enumerate(val):
        cv2.imwrite(path+"val/Val-"+str(i)+".png",imagen)
        f.write(str(y_val[i])+"\n")
    f.close()

Función que permite el entrenamiento y la clasificación utilizando un clasificador que se pasa como parámetro, además los datos pueden ser normalizados si el parámetro norm tiene el valor True. También permite utilizar diferentes valores de score con los que seleccionar el conjunto de características de los datos que se pasan como parámetros. Devuelve el tiempo de ejecución así como los valores de accuracy obtenidos en train, validación y test.

In [7]:
def clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[0], norm=False):

    #Normalización de los datos:
    if(norm == True):
        Xtrain = Xtrain/255
        Xtest = Xtest/255
        Xval = Xval/255

    accTrain=[]
    accTest=[]
    accVal=[]
    #Comenzar a contar el tiempo de ejecución:
    start_time = time.time()

    for i,k in enumerate(score):
        #Seleccionar aquellos pixeles que tengan un Score mayor o igual al establecido
        pixelesSelec = df[df.Score >= k]['Pixeles'].values.tolist()

        #Entrenar el modelo con los ejemplos con el nuevo numero de caracteristicas:
        #Generar el nuevo conjunto de Train:
        X_train_tratada = Xtrain[:,pixelesSelec]

        #Entrenar el modelo:
        clasificador.fit(X_train_tratada,y_train)

        #Obtener la predcción en train y el accuracy:
        predictTrain = clasificador.predict(X_train_tratada)
        accTrain.append(metrics.accuracy_score(predictTrain,y_train)*100)

        #Generar el nuevo conjunto de Val:
        X_val_tratada = Xval[:,pixelesSelec]

        #Obtener la predcción en val y el accuracy:
        predictVal = clasificador.predict(X_val_tratada)
        accVal.append(metrics.accuracy_score(predictVal,y_val)*100)

        #Generar el nuevo conjunto de Test:
        X_test_tratada = Xtest[:,pixelesSelec]

        #Obtener la predcción en test y el accuracy:
        predictTest = clasificador.predict(X_test_tratada)
        accTest.append(metrics.accuracy_score(predictTest,y_test)*100)

        print('El rendimiento en entrenamiento con {} variables para un score de {} es de {}%'.format(len(pixelesSelec),k,accTrain[i]))
        print('El rendimiento en validacion con {} variables para un score de {}  es de {}%'.format(len(pixelesSelec),k,accVal[i]))
        print('El rendimiento en test con {} variables para un score de {}  es de {}%'.format(len(pixelesSelec),k,accTest[i]))

    tiempo_ejecucion = time.time() - start_time
    print("--- %s seconds ---" % (tiempo_ejecucion))

Esta función genera la matriz de confusión para un conjunto de datos de entrada y muestra los valores de precisión, recall y f-score

In [8]:
def genera_confusion(clasificador,X,y):
    print("********************************* Plot de matriz de confusión *********************************")
    class_names = ['COVID-19','Normal','Viral']
    titles_options = [("Matriz de confusion sin normalizar", None),
                      ("Matriz de confusion normalizada", 'true')]
    for title, normalize in titles_options:
        disp = metrics.plot_confusion_matrix(clasificador, X, y,
                                     display_labels=class_names,
                                     cmap=plt.cm.Blues,
                                     normalize=normalize)
        disp.ax_.set_title(title)
        print(title)
        print(disp.confusion_matrix)
    plt.show()
    print("********************************* Valores de precisión, recall y f-score *********************************")
    print(metrics.classification_report(y, clasificador.predict(X)))

Función que permite copiar ficheros entre directorios

In [14]:
def copiaImagenes(src='ImagenesCovidExtra/', dst='Data/COVID-19/'):
    if(not os.path.exists(src)):
        print("¡Error! El directorio de origen no existe")
        return
    elif(not os.path.exists(src)):
        print("¡Error! El directorio de destino no existe")
        return
    print("Copiando ficheros de "+src+" a "+dst)
    src_files = os.listdir(src)
    for file_name in src_files:
        full_file_name = os.path.join(src, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, dst)

Función que permite eliminar las imágenes extra de COVID

In [36]:
def eliminaImagenesExtra(src='ImagenesCovidExtra/', dst='Data/COVID-19/'):
    if(os.path.exists(src)):
        if(os.path.exists(dst)):
            print("Eliminando las imágenes extra de: "+dst)
            
            listaImagenes = os.listdir(src)
            for imagen in listaImagenes:
                fichero = dst+imagen
                if os.path.exists(fichero):
                    os.remove(fichero)
        else:
            print("Path de destino no encontrado "+dst)
    else:
         print("Path de origen no encontrado "+src)

Celda principal para la ejecución de las pruebas de la primera aproximación

In [ ]:
#Constantes del problema:
pathCovid = "Data/COVID-19/"
pathNormal = "Data/NORMAL/"
pathViral = "Data/Viral Pneumonia/" 
pathImagenesExtra = "ImagenesCovidExtra/"
pathImagenes= "Dataset/"
fichero_train = "train.txt"
fichero_val = "val.txt"
fichero_test = "test.txt"
dx=256 #Tamaño de las imágenes en eje x
dy=256 #Tamaño de las imágenes en eje y
np.random.seed(0)
###############################################################################################################################

#Pretratamiento de imagenes y almacenamiento de las mismas:
#Realizamos el tratamiento de las imágenes:
listaCOVID = tratamiento(pathCovid,dx,dy)
listaNORMAL = tratamiento(pathNormal,dx,dy)
listaVIRAL = tratamiento(pathViral,dx,dy)

#Generamos el particionamiento
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
train, test, val, y_train, y_test, y_val = particionamiento(listaDatos)

#Almacenamos las imágenes en disco
almacenaImagenes(train,test,val,y_train,y_test,y_val,pathImagenes)
###############################################################################################################################

#Lectura de las imágenes desde disco:
train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

###############################################################################################################################

#Dejamos las imágenes en 2D para la selección de características:
Xtrain = np.reshape(train, (train.shape[0], dx*dy))
Xtest = np.reshape(test, (test.shape[0], dx*dy))
Xval = np.reshape(val, (val.shape[0], dx*dy))

#Comprobación de las características y selección:
Kbest = SelectKBest(f_classif) #Generamos el objeto SelectKBest
Kbest.fit(Xtrain, y_train) #Entrenar el modelo
numPixel = list(range(dx*dy)) #Generar el listado de número de píxeles

#Generar un dataframe donde la primera columna sean el número de pixeles o características y la segundo el score:
df = pd.DataFrame({'Pixeles': numPixel,'Score': Kbest.scores_.tolist()})
#Generar un plot de barras para mostrar la información:
plot = df.Score.plot(kind = 'hist', bins=40, title='Selección de Kbest')
plot.set_xlabel("Score")
plot.set_ylabel("Frecuencia")

###############################################################################################################################

############################################ Pruebas con Regresión logística ##################################################
###############################################################################################################################
###############################################################################################################################

#Probamos a clasificar con todas las características y con lo parámetros por defecto del clasificador:
print("********************************* Prueba clasificador con todas las características ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[0], norm=False)

#Probamos a clasificar normalizando los datos de entrada:
print("********************************* Prueba clasificador con todas las características normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[0], norm=True)

#Probamos con diferentes valores de score:
print("********************************* Prueba clasificador con varios valores de score ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100,200,300,450], norm=False)

#Probamos con diferentes valores de score y normalizando los datos:
print("********************************* Prueba clasificador con varios valores de score normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100,200,300,450], norm=True)

#Probamos a calcular los tiempos de ejecución con score 100 normalizando los datos de entrada:
print("********************************* Prueba clasificador tiempo ejecución con score 100 normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100], norm=True)

#Probamos a calcular los tiempos de ejecución con score 200 normalizando los datos de entrada:
print("********************************* Prueba clasificador tiempo ejecución con score 200 normalizandoo ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)

# #Probamos a calcular los tiempos de ejecución con score 300 normalizando los datos de entrada:
print("********************************* Prueba clasificador tiempo ejecución con score 300 normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[300], norm=True)

# #Probamos a calcular los tiempos de ejecución con score 450 normalizando los datos de entrada:
print("********************************* Prueba clasificador tiempo ejecución con score 450 normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[450], norm=True)

#Probamos para un valor de score de 200 y generamos las matrices de confusión:
print("********************************* Prueba clasificador con score 200 y normalizando ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
X_val_tratada = Xval[:,pixelesSelec]
X_test_tratada = Xtest[:,pixelesSelec]
genera_confusion(clasificador,X_train_tratada,y_train)
genera_confusion(clasificador,X_val_tratada,y_val)
genera_confusion(clasificador,X_test_tratada,y_test)

#Hacemos un gridSearch para encontrar los mejores valores de los parámetros para la regresión logística:
print("********************************* GridSearch para score 200 y diferentes algoritmos de optimización y normalización ********************************* ")
params={'solver':['newton-cg','sag', 'saga', 'lbfgs'], 'penalty':['l1', 'l2', 'elasticnet'], 'C':[1,10,100,1000]}
cv=5
clf = GridSearchCV(LogisticRegression(), params, cv=cv, scoring='accuracy', n_jobs=-1)
pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
clf.fit(X_train_tratada,y_train)
print("EL mejor resultado obtenido ha sido: "+str(clf.best_score_)+", los mejores parámetros obtenidos son: "+str(clf.best_params_))
clasificador = LogisticRegression(solver='saga',penalty="l2",C=100,n_jobs=-1)
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
X_val_tratada = Xval[:,pixelesSelec]
X_test_tratada = Xtest[:,pixelesSelec]
genera_confusion(clasificador,X_train_tratada,y_train)
genera_confusion(clasificador,X_val_tratada,y_val)
genera_confusion(clasificador,X_test_tratada,y_test)

#Hacemos otros gridSearch de nuevo pero para liblinear ya que hemos comprobado que es la mejor función de optimización para este problema:
print("********************************* GridSearch para score 200 y diferentes algoritmos de normalización  ********************************* ")
params={'solver':['liblinear'], 'penalty':['l1', 'l2', 'elasticnet'], 'C':[2,5,10,100]}
clf = GridSearchCV(LogisticRegression(), params, cv=cv, scoring='accuracy', n_jobs=-1)
clf.fit(X_train_tratada,y_train)
print("EL mejor resultado obtenido ha sido: "+str(clf.best_score_)+", los mejores parámetros obtenidos son: "+str(clf.best_params_))
clasificador = LogisticRegression(solver='liblinear',penalty="l1",C=5)
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
X_val_tratada = Xval[:,pixelesSelec]
X_test_tratada = Xtest[:,pixelesSelec]
genera_confusion(clasificador,X_train_tratada,y_train)
genera_confusion(clasificador,X_val_tratada,y_val)
genera_confusion(clasificador,X_test_tratada,y_test)

#Añadimos nuevos ejemplos para COVID-19 y probamos con la mejor configuración obtenida:
print("********************************* Prueba clasificador con más ejemplos de COVID-19 ********************************* ")
#Copiar las nuevas imagenes al directorio inicial de covid para tratarlas:
copiaImagenes(pathImagenesExtra,pathCovid)
#Pretratamiento de imagenes y almacenamiento de las mismas:
#Realizamos el tratamiento de las imágenes:
listaCOVID = tratamiento(pathCovid,dx,dy)
listaNORMAL = tratamiento(pathNormal,dx,dy)
listaVIRAL = tratamiento(pathViral,dx,dy)

#Generamos el particionamiento
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
train, test, val, y_train, y_test, y_val = particionamiento(listaDatos)

#Almacenamos las imágenes en disco
almacenaImagenes(train,test,val,y_train,y_test,y_val,pathImagenes)
###############################################################################################################################

#Lectura de las imágenes desde disco:
train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

Xtrain = np.reshape(train, (train.shape[0], dx*dy))
Xtest = np.reshape(test, (test.shape[0], dx*dy))
Xval = np.reshape(val, (val.shape[0], dx*dy))

#Comprobación de las características y selección:
Kbest = SelectKBest(f_classif) #Generamos el objeto SelectKBest
Kbest.fit(Xtrain, y_train) #Entrenar el modelo
numPixel = list(range(dx*dy)) #Generar el listado de número de píxeles

#Generar un dataframe donde la primera columna sean el número de pixeles o características y la segundo el score:
df = pd.DataFrame({'Pixeles': numPixel,'Score': Kbest.scores_.tolist()})

###############################################################################################################################
clasificador = LogisticRegression(solver='liblinear')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
X_val_tratada = Xval[:,pixelesSelec]
X_test_tratada = Xtest[:,pixelesSelec]
genera_confusion(clasificador,X_train_tratada,y_train)
genera_confusion(clasificador,X_val_tratada,y_val)
genera_confusion(clasificador,X_test_tratada,y_test)

eliminaImagenesExtra(pathImagenesExtra,pathCovid)
############################################ Fin de pruebas con Regresión logística ###########################################
###############################################################################################################################
###############################################################################################################################

############################################ Pruebas con Support Vector Machines ##############################################
###############################################################################################################################
###############################################################################################################################

#Pretratamiento de imagenes y almacenamiento de las mismas:
#Realizamos el tratamiento de las imágenes:
listaCOVID = tratamiento(pathCovid,dx,dy)
listaNORMAL = tratamiento(pathNormal,dx,dy)
listaVIRAL = tratamiento(pathViral,dx,dy)

#Generamos el particionamiento
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
train, test, val, y_train, y_test, y_val = particionamiento(listaDatos)

#Almacenamos las imágenes en disco
almacenaImagenes(train,test,val,y_train,y_test,y_val,pathImagenes)
###############################################################################################################################

#Lectura de las imágenes desde disco:
train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

###############################################################################################################################

#Dejamos las imágenes en 2D para la selección de características:
Xtrain = np.reshape(train, (train.shape[0], dx*dy))
Xtest = np.reshape(test, (test.shape[0], dx*dy))
Xval = np.reshape(val, (val.shape[0], dx*dy))

#Comprobación de las características y selección:
Kbest = SelectKBest(f_classif) #Generamos el objeto SelectKBest
Kbest.fit(Xtrain, y_train) #Entrenar el modelo
numPixel = list(range(dx*dy)) #Generar el listado de número de píxeles

#Generar un dataframe donde la primera columna sean el número de pixeles o características y la segundo el score:
df = pd.DataFrame({'Pixeles': numPixel,'Score': Kbest.scores_.tolist()})

#Hacemos un gridSearch para encontrar los mejores valores de los parámetros para la SVC
print("********************************* GridSearch para score 100 y SVC y normalización ********************************* ")
pixelesSelec = df[df.Score >= 100]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
gs_clf_svm = model_selection.GridSearchCV(estimator=svm.SVC(), param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')}
                                          ,return_train_score=True, n_jobs = -1)
gs_clf_svm.fit(X_train_tratada,y_train)
print("EL mejor resultado obtenido ha sido: "+str(gs_clf_svm.best_score_)+", los mejores parámetros obtenidos son: "+str(gs_clf_svm.best_params_))
clasificador = svm.SVC(gs_clf_svm.best_score_)
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100], norm=True)

#Adaboost para LR
print("********************************* Prueba clasificador Adaboost para LR ********************************* ")
LR = LogisticRegression(solver='liblinear')
adaBoostLR = AdaBoostClassifier(base_estimator=LR, n_estimators= 4)
clasifica(adaBoostLR, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100], norm=True)

#Adaboost para Tree
print("********************************* Prueba clasificador Adaboost para Tree ********************************* ")
adaBoost = AdaBoostClassifier(n_estimators= 100)
clasifica(adaBoost, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100], norm=True)

#OVR
print("********************************* Prueba clasificador OVR ********************************* ")
LogReg = LogisticRegression(solver='liblinear', C=10, penalty='l1')
OVR = OneVsRestClassifier(LogReg).fit(X_train_tratada,y_train)
clasifica(OVR, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[100], norm=True)

#Hacemos un gridSearch para encontrar los mejores valores de los parámetros para la SVC
print("********************************* GridSearch para score 200 y SVC y normalización ********************************* ")
pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
X_test_tratada = Xtest[:,pixelesSelec]
X_val_tratada = Xval[:,pixelesSelec]
clasificador = svm.SVC(C=10, kernel = 'rbf')
clasifica(clasificador, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
genera_confusion(clasificador,X_train_tratada,y_train)
genera_confusion(clasificador,X_val_tratada,y_val)
genera_confusion(clasificador,X_test_tratada,y_test)

#Adaboost para LR
print("********************************* Prueba clasificador Adaboost para LR ********************************* ")
LR = LogisticRegression(solver='liblinear')
adaBoostLR = AdaBoostClassifier(base_estimator=LR, n_estimators= 4)
clasifica(adaBoostLR, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
genera_confusion(adaBoostLR,X_train_tratada,y_train)
genera_confusion(adaBoostLR,X_val_tratada,y_val)
genera_confusion(adaBoostLR,X_test_tratada,y_test)

#Adaboost para Tree
print("********************************* Prueba clasificador Adaboost para Tree ********************************* ")
adaBoost = AdaBoostClassifier(n_estimators= 100)
clasifica(adaBoost, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
genera_confusion(adaBoost,X_train_tratada,y_train)
genera_confusion(adaBoost,X_val_tratada,y_val)
genera_confusion(adaBoost,X_test_tratada,y_test)

#OVR
print("********************************* Prueba clasificador OVR ********************************* ")
LogReg = LogisticRegression(solver='liblinear', C=10, penalty='l1')
OVR = OneVsRestClassifier(LogReg).fit(X_train_tratada,y_train)
clasifica(OVR, df, Xtrain, Xtest, Xval, y_train, y_test, y_val, score=[200], norm=True)
genera_confusion(OVR,X_train_tratada,y_train)
genera_confusion(OVR,X_val_tratada,y_val)
genera_confusion(OVR,X_test_tratada,y_test)

############################################ Fin de pruebas con SVM ###########################################################
###############################################################################################################################
###############################################################################################################################

# Segunda aproximación del problema

Función que permite obtener las características de HOG para cada imagen del conjunto de datos

In [3]:
def obtener_caracteristicas_hog(X,numFeatures):
    n = X.shape[0]
    
    hog_descriptor = np.empty((n,numFeatures))
    for i in range(0,n):
        fd = hog(X[i], orientations=8, pixels_per_cell=(16, 16),cells_per_block=(1, 1), visualize=False, multichannel=False)
        hog_descriptor[i] = fd.reshape(1,-1)
    return hog_descriptor

Función que permite el entrenamiento y la clasificación utilizando un clasificador que se pasa como parámetro, además los datos pueden ser normalizados si el parámetro norm tiene el valor True. Devuelve el tiempo de ejecución así como los valores de accuracy obtenidos en train, validación y test.

In [4]:
def clasifica_segunda(clasificador, Xtrain, Xtest, Xval, y_train, y_test, y_val):
    
    accTrain=[]
    accTest=[]
    accVal=[]
    start_time = time.time()

    #Entrenar el modelo:
    clasificador.fit(Xtrain,y_train)

    #Obtener la predcción en train y el accuracy:
    predictTrain = clasificador.predict(Xtrain)
    accTrain.append(metrics.accuracy_score(predictTrain,y_train)*100)

    #Obtener la predcción en val y el accuracy:
    predictVal = clasificador.predict(Xval)
    accVal.append(metrics.accuracy_score(predictVal,y_val)*100)

    #Obtener la predcción en test y el accuracy:
    predictTest = clasificador.predict(Xtest)
    accTest.append(metrics.accuracy_score(predictTest,y_test)*100)

    print('El rendimiento en entrenamiento es de {}%'.format(accTrain[0]))
    print('El rendimiento en validacion es de {}%'.format(accVal[0]))
    print('El rendimiento en test es de {}%'.format(accTest[0]))

    tiempo_ejecucion = time.time() - start_time
    print("--- %s seconds ---" % (tiempo_ejecucion))

Celda principal para la ejecución de las pruebas de la segunda aproximación

In [ ]:
#Constantes del problema 
pathCovid = "Data/COVID-19/"
pathNormal = "Data/NORMAL/"
pathViral = "Data/Viral Pneumonia/" 
pathImagenesExtra = "ImagenesCovidExtra/"
pathImagenes= "Dataset/"
fichero_train = "train.txt"
fichero_val = "val.txt"
fichero_test = "test.txt"
dx=256 #Tamaño de las imágenes en eje x
dy=256 #Tamaño de las imágenes en eje y
numFeatures = 2048
METHOD = 'uniform'
radius = 3
n_points = 8 * radius
c = [2,5,10,100]
np.random.seed(0)

train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

#Extraer Las características con HOG:
X_train = obtener_caracteristicas_hog(train)
X_val = obtener_caracteristicas_hog(val)
X_test = obtener_caracteristicas_hog(test)

############################################ Pruebas con HOG ##################################################################
###############################################################################################################################
###############################################################################################################################

# Probamos a clasificar con todas las características y con lo parámetros por defecto del clasificador:
print("********************************* Prueba clasificador con todas las características de HOG ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica_segunda(clasificador, X_train, X_test, X_val, y_train, y_test, y_val)
genera_confusion(clasificador,X_train,y_train)
genera_confusion(clasificador,X_test,y_test)
genera_confusion(clasificador,X_val,y_val)

# Añadimos nuevos ejemplos para COVID-19 y probamos con la mejor configuración obtenida:
#Copiar las nuevas imagenes al directorio inicial de covid para tratarlas:
copiaImagenes(pathImagenesExtra,pathCovid)
#Pretratamiento de imagenes y almacenamiento de las mismas:
#Realizamos el tratamiento de las imágenes:
listaCOVID = tratamiento(pathCovid,dx,dy)
listaNORMAL = tratamiento(pathNormal,dx,dy)
listaVIRAL = tratamiento(pathViral,dx,dy)

#Generamos el particionamiento
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
train, test, val, y_train, y_test, y_val = particionamiento(listaDatos)

#Almacenamos las imágenes en disco
almacenaImagenes(train,test,val,y_train,y_test,y_val,pathImagenes)
###############################################################################################################################

#Lectura de las imágenes desde disco:
train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)
###############################################################################################################################

#Extraer Las características con HOG:
X_train = obtener_caracteristicas_hog(train,numFeatures)
X_val = obtener_caracteristicas_hog(val,numFeatures)
X_test = obtener_caracteristicas_hog(test,numFeatures)

print("********************************* Prueba clasificador con más ejemplos de COVID-19 ********************************* ")
clasificador = LogisticRegression(solver='liblinear')
clasifica_segunda(clasificador, X_train, X_test, X_val, y_train, y_test, y_val)
genera_confusion(clasificador,X_train,y_train)
genera_confusion(clasificador,X_test,y_test)
genera_confusion(clasificador,X_val,y_val)

print("********************************* Prueba clasificador con regularización para más ejemplos de COVID-19 ********************************* ")
clasificador = LogisticRegression(solver='liblinear',C=5,penalty="l1")
clasifica_segunda(clasificador, X_train, X_test, X_val, y_train, y_test, y_val)
genera_confusion(clasificador,X_train,y_train)
genera_confusion(clasificador,X_test,y_test)
genera_confusion(clasificador,X_val,y_val)

#Probamos a utilizar kbest de nuevo para ver si mejoramos los resultados que hemos obtenido:
print("********************************* Prueba clasificador con menor número de características ********************************* ")
#Comprobación de las características y selección:
Kbest = SelectKBest(f_classif) #Generamos el objeto SelectKBest
Kbest.fit(X_train, y_train) #Entrenar el modelo
numPixel = list(range(numFeatures)) #Generar el listado de número de píxeles

#Generar un dataframe donde la primera columna sean el número de características y la segundo el score:
df = pd.DataFrame({'Pixeles': numPixel,'Score': Kbest.scores_.tolist()})
#Generar un plot de barras para mostrar la información:
plot = df.Score.plot(kind = 'hist', bins=40, title='Selección de Kbest')
plot.set_xlabel("Score")
plot.set_ylabel("Frecuencia")
scores=[25,50,75,100,150,200]
for score in scores:
    clasificador = LogisticRegression(solver='liblinear')
    clasifica(clasificador, df, X_train, X_test, X_val, y_train, y_test, y_val, score=[score], norm=False)
    pixelesSelec = df[df.Score >= score]['Pixeles'].values.tolist()
    X_train_tratada = X_train[:,pixelesSelec]
    X_val_tratada = X_val[:,pixelesSelec]
    X_test_tratada = X_test[:,pixelesSelec]
    genera_confusion(clasificador,X_train_tratada,y_train)
    genera_confusion(clasificador,X_val_tratada,y_val)
    genera_confusion(clasificador,X_test_tratada,y_test)

#Probamos a utilizar kbest de nuevo para ver si mejoramos los resultados que hemos obtenido:
print("********************************* Prueba clasificador OVR con HOG y más ejemplos COVID-19 ********************************* ")
LogReg = LogisticRegression(solver='liblinear')
OVR = OneVsRestClassifier(LogReg).fit(X_train,y_train)
clasifica_segunda(OVR, X_train, X_test, X_val, y_train, y_test, y_val)
genera_confusion(OVR,X_train,y_train)
genera_confusion(OVR,X_val,y_val)
genera_confusion(OVR,X_test,y_test)
eliminaImagenesExtra(pathImagenesExtra,pathCovid)

############################################ Fin de pruebas con HOG ###########################################################
###############################################################################################################################
###############################################################################################################################

############################################ Pruebas con LBP ##################################################################
###############################################################################################################################
###############################################################################################################################

#Pretratamiento de imagenes y almacenamiento de las mismas:
#Realizamos el tratamiento de las imágenes:
listaCOVID = tratamiento(pathCovid,dx,dy)
listaNORMAL = tratamiento(pathNormal,dx,dy)
listaVIRAL = tratamiento(pathViral,dx,dy)

#Generamos el particionamiento
listaDatos = [listaCOVID, listaNORMAL, listaVIRAL] 
train, test, val, y_train, y_test, y_val = particionamiento(listaDatos)

#Almacenamos las imágenes en disco
almacenaImagenes(train,test,val,y_train,y_test,y_val,pathImagenes)
###############################################################################################################################

#Lectura de las imágenes desde disco:
train, y_train = lectura("train/",fichero_train,pathImagenes)
val, y_val = lectura("val/",fichero_val,pathImagenes)
test, y_test = lectura("test/",fichero_test,pathImagenes)

#Comprobación de las características y selección:
Kbest = SelectKBest(f_classif) #Generamos el objeto SelectKBest
Kbest.fit(Xtrain, y_train) #Entrenar el modelo
numPixel = list(range(dx*dy)) #Generar el listado de número de píxeles

#Generar un dataframe donde la primera columna sean el número de pixeles o características y la segundo el score:
df = pd.DataFrame({'Pixeles': numPixel,'Score': Kbest.scores_.tolist()})

lbpTrain = local_binary_pattern(np.reshape(train, (train.shape[0], dx*dy)), n_points, radius, METHOD)
lbpTest = local_binary_pattern(np.reshape(test, (test.shape[0], dx*dy)), n_points, radius, METHOD)
lbpVal = local_binary_pattern(np.reshape(val, (val.shape[0], dx*dy)), n_points, radius, METHOD)

pixelesSelec = df[df.Score >= 200]['Pixeles'].values.tolist()
X_train_tratada = Xtrain[:,pixelesSelec]
X_test_tratada = Xtest[:,pixelesSelec]
X_val_tratada = Xval[:,pixelesSelec]

for x in c:
    print("********************************* Prueba Regresion logistica ********************************* ")
    LogReg = LogisticRegression(solver='liblinear', C=x, penalty='l1')
    clasifica(LogReg, df, lbpTrain, lbpTest, lbpVal, y_train, y_test, y_val, score=[200], norm=True)
    print("********************************* Prueba clasificador OVR ********************************* ")
    OVR = OneVsRestClassifier(LogReg)
    clasifica(OVR, df, lbpTrain, lbpTest, lbpVal, y_train, y_test, y_val, score=[200], norm=True)
#Miramos la matriz de confusion para c=5
LogReg = LogisticRegression(solver='liblinear', C=5, penalty='l1')
clasifica(LogReg, df, lbpTrain, lbpTest, lbpVal, y_train, y_test, y_val, score=[200], norm=True)#OVR
OVR = OneVsRestClassifier(LogReg)
clasifica(OVR, df, lbpTrain, lbpTest, lbpVal, y_train, y_test, y_val, score=[200], norm=True)
genera_confusion(LogReg,X_train_tratada,y_train)
genera_confusion(LogReg,X_val_tratada,y_val)
genera_confusion(LogReg,X_test_tratada,y_test)
genera_confusion(OVR,X_train_tratada,y_train)
genera_confusion(OVR,X_val_tratada,y_val)
genera_confusion(OVR,X_test_tratada,y_test)

############################################ Fin de pruebas con LBP ###########################################################
###############################################################################################################################
###############################################################################################################################